In [1]:
!pip install ultralytics

     -------------------------------------- 756.5/756.5 kB 3.2 MB/s eta 0:00:00


In [15]:
'''
#將.xml轉成Yolo所需 .txt檔案
import os
import xml.etree.ElementTree as ET

def convert_coordinates(size, box):
    # 轉換坐標
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders):
    # 解析XML並轉換為YOLO格式
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for output_folder in output_folders:
        folder_name = os.path.basename(output_folder)
        output_dir = os.path.join(output_root, folder_name)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_path))[0] + '.txt')
        with open(output_file_path, 'w') as f:
            for obj in root.findall('object'):
                cls = obj.find('name').text
                if cls not in class_dict:
                    continue
                cls_id = class_dict[cls]
                xml_box = obj.find('bndbox')
                box = (float(xml_box.find('xmin').text), float(xml_box.find('xmax').text), 
                       float(xml_box.find('ymin').text), float(xml_box.find('ymax').text))
                bb = convert_coordinates((w,h), box)
                f.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")

# 資料夾列表
folders = ["訓練集_xml", "驗證集_xml", "測試集_xml"]
o_folders = ["訓練集_txt", "驗證集_txt", "測試集_txt"]

# 分類字典，將類別名映射到整數標籤
class_dict = {"container": 0}

# 輸出根資料夾路徑
output_root = "E:\Downloads\貨櫃資料集"

# 如果輸出根資料夾不存在，則創建
if not os.path.exists(output_root):
    os.makedirs(output_root)

# 迴圈處理每個資料夾
for folder, o_folder in zip(folders, o_folders):
    folder_path = os.path.join("E:\Downloads\貨櫃資料集", folder)  # 資料夾路徑
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]  # 獲取所有XML檔案
    output_folders = [os.path.join(output_root, o_folder)]  # 輸出資料夾路徑
    for xml_file in xml_files:
        xml_path = os.path.join(folder_path, xml_file)  # XML檔案路徑
        convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders)
'''

In [1]:
from ultralytics import YOLO  # 導入YOLO模型

# 載入模型
model = YOLO("E:\Downloads\影片資料集\yolov8n.yaml")  # 從頭構建一個新模型
model = YOLO("yolov8n.pt")  # 載入預訓練模型（推薦用於訓練）

# 使用模型
model.train(data="coco8.yaml", epochs=3)  # 訓練模型
metrics = model.val()  # 在驗證集上評估模型性能
results = model("https://ultralytics.com/images/bus.jpg")  # 對圖片進行預測

Ultralytics YOLOv8.2.11  Python-3.9.13 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning C:\Users\USER\datasets\coco8\labels\train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████|
val: Scanning C:\Users\USER\datasets\coco8\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train6
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3     0.814G      1.069       3.51      1.514         21        640: 100%|██████████| 1/1 [00:01<00:00,  1.97
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0

                   all          4         17       0.62      0.878      0.888      0.615


...


3 epochs completed in 0.003 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.5MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.5MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics YOLOv8.2.11  Python-3.9.13 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 12288MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all          4         17      0.571      0.833      0.872       0.62
                person          4         10      0.644        0.5      0.509      0.282
                   dog          4          1      0.318          1      0.995      0.597
                 horse          4          2      0.693          1      0.995      0.648
              elephant          4          2      0.387        0.5      0.745      0.304
              umbrella          4          1      0.541          1      0.995      0.995
          potted plant          4          1      0.844          1      0.995      0.895
Speed: 0.3ms preprocess, 3.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train6
Ultralytics YOLOv8.2.11  Python-3.9.13 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 12288MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning C:\Users\USER\datasets\coco8\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:06<0


                   all          4         17      0.593      0.882      0.886      0.625
                person          4         10      0.638        0.5      0.509      0.282
                   dog          4          1      0.304          1      0.995      0.597
                 horse          4          2      0.683          1      0.995      0.648
              elephant          4          2      0.597      0.792      0.828      0.333
              umbrella          4          1      0.537          1      0.995      0.995
          potted plant          4          1      0.796          1      0.995      0.895
Speed: 1.3ms preprocess, 23.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train62

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\USER\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 480)
Ultralytics YOLO

In [6]:
for result in results:
    result.show()